In [ ]:
Objective : To extract features decided and groups on unseen data for difining their importance and to identify are we still missing any features .

Conclusion - Sentence transformers worked best among bert2vec, word2vec, ollama. 

Approach : Remove noise, punctuation marks & stop words,
Features created are total 110 in count and are standardized to bring in format that is available in comments.Sentence level tokenization.Within sentence tokenization , word toekenization was done to create a list of word tokens for each sentence.
Features created are total 110 in count and are standardized to bring in format that is available in comments.Sentence level tokenization.Within sentence tokenization , word toekenization was done to create a list of word tokens for each sentence.
These features are standardized or written in order which is present in comments.Like customer - biography/work/profile - location ,customer - biography/work/profile - date ,customer - biography/work/profile - salary 
Training word2vec model for generating embeddings for each senetence by taking avg of word embeddings.
So pick every sentence embeddings and check cosine similarity with every feature. Set a threshold value based on manual obeservation , now for feature
with cosine similarity greater than threshold ,determine the category they fall into.
Solution : ID , DOB , Occupation , Account & Transaction , PEP , Name ,Gender.



In [159]:
import nltk
# nltk.download("punkt")

In [160]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import re
from nltk.tokenize import sent_tokenize, word_tokenize
# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [161]:
import calendar
cal=list(calendar.month_name)
for i in list(calendar.month_name[1:]):
    cal.append(i[:3])

In [162]:
df=pd.read_excel("Additional words in topics tf idf bi tri.xlsx")

KeyboardInterrupt: 

In [ ]:
df.drop(["Unnamed: 0.1","Unnamed: 0",],axis=1,inplace=True)
df.dropna(inplace=True)

In [165]:
df_samp

,index,V_COMMENTS,Counts,V_COMMENTS_2,Tokens,Match_Mismatch,V_COMMENTS_PROCESSED,V_COMMENTS_TOKENS,V_COMMENTS_3,Vect,...,Features_present_cosine_top_5,Features_present_cosine_actual_anchors,Feat_Categories,Categories,Features_present_J,Features_present_Top_5,Features_actual_J_anchors,Feat_Categories_J,Categories_J,Rel_flag
0,21675,mismatch dob. customer dob does not fall withi...,2,mismatch dob customer dob fall within watch p...,"[mismatch, dob, customer, dob, fall, within, w...",[mismatch],dob . customer dob fall within watch person po...,"[dob, customer, dob, fall, within, watch, pers...","[[dob], [customer, dob, fall, within, watch, p...","[[-0.7280587, -1.7479811, 3.5171788, -0.176680...",...,"[[0.93859226, [date, birth, dob, fall, within,...",[date birth dob fall within watch person dob r...,[(date birth dob fall within watch person dob ...,"[DOB, Date of Birth (DOB)]","[[0.3333333333333333, [date, birth, dob], [dob...","[[0.6, [date, birth, dob, fall, within, watch,...","[date birth dob, watch person age deceased, wa...","[(date birth dob, Date of Birth (DOB)), (watch...","[Deceased Status, DOB, Date of Birth (DOB)]",False
1,16782,mismatch profile - customer as manufacturing l...,2,mismatch profile customer manufacturing labour...,"[mismatch, profile, customer, manufacture, lab...",[mismatch],profile customer manufacturing labourers per u...,"[profile, customer, manufacturing, labourers, ...","[[profile, customer, manufacturing, labourers,...","[[0.729185, -0.71218747, -2.4682145, 0.8441992...",...,"[[0.5984235, [customer, joint, account, flag],...",[customer joint account flag],"[(customer joint account flag, Account &Transa...",[Account &Transaction],"[[0.11764705882352941, [customer, occupation, ...","[[0.11764705882352941, [customer, occupation, ...","[customer marital status, customer deceased da...","[(customer marital status, UNKOWN/Miscellanoeo...","[Biography/ Occupation, UNKOWN/Miscellanoeous,...",NaN
2,142022,mismatch profile. customer who is manufacturin...,1,mismatch profile customer manufacturing labou...,"[mismatch, profile, customer, manufacturing, l...",[mismatch],profile . customer manufacturing labourers fla...,"[profile, customer, manufacturing, labourers, ...","[[profile], [customer, manufacturing, labourer...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...",...,"[[0.8977755, [customer, occupation, biography,...","[watch person uncle occupation profile, watch ...","[(watch person uncle occupation profile, Relat...",[Biography/ Occupation],"[[0.25, [customer, occupation, biography, prof...","[[0.25, [customer, occupation, biography, prof...","[watch person spouse occupation profile, watch...","[(watch person spouse occupation profile, Biog...",[Biography/ Occupation],False
3,177912,false hit. watchperson is son of pep who is a ...,1,watchperson son pep member pyithu hluttaw repr...,"[watchperson, son, pep, member, pyithu, hlutta...",[false hit],watchperson son pep member pyithu hluttaw repr...,"[watchperson, son, pep, member, pyithu, hlutta...","[[watchperson, son, pep, member, pyithu, hlutt...","[[2.7791328, 0.58117193, 0.23717156, 0.3814372...",...,"[[0.61741924, [watch, person, son, pep, flag],...","[watch person father law pep flag, watch perso...","[(watch person father law pep flag, PEP), (wat...","[PEP, Account &Transaction]","[[0.15384615384615385, [watch, person, son, pe...","[[0.25, [customer, job, work, location], [cust...","[customer job work location, customer job work...","[(customer job work location, Biography/ Occup...","[PEP, Related Parties, Biography/ Occupation]",NaN
4,50402,mismatch profile based on host the customer is...,1,mismatch profile based host customer managing ...,"[mismatch, profile, base, host, customer, mana...",[mismatch],profile based host customer managing directors...,"[profile, based, host, customer, managing, dir...","[[profile, based, host, customer, managing, di...","[[-0.1911412, -0.13274044, 0.37196544, -0.7015...",...,"[[0.80112, [customer, cif, creat

In [6]:
# sequence=df[df["V_COMMENTS"].apply(lambda x: len(x)>100)]["V_COMMENTS"].loc[221569]

In [209]:
df_features=pd.read_csv("Topic_Feature_List_Agg_brd_labs.csv")

In [8]:
df.drop(["id_2_feature","TFIDF_Feature","Topic_tfidf_lda","All_Topic_tfidf_Probabilities","TF_tfidf_Topic_No"," TF IDF Topic words"],axis=1,inplace=True)

In [210]:
df_features

,S.no,Standardized_Anchors_BRD_LABS,BRD_FLAG,Categories,Category_labs
0,1,Word count difference,Y,Full Legal Name - Word Count,NaN
1,2,Name structure difference,Y,Full Legal Name - Name Structure,NaN
2,3,Name does not sound similiar,Y,Full Legal Name - Name Does Not Sound Similar,NaN
3,4,Native name charachter difference,Y,Full Legal Name - Native Name,Name
4,5,NaN,Y,Category (Vessel vs. Individual vs. Entity),NaN
...,...,...,...,...,...
430,431,Customer - Transaction - Number of transactions,NaN,Account & transactions,NaN
431,432,Customer - Transaction - Amount,NaN,Account & transactions,NaN
432,433,Customer - Transaction - Online / offline,NaN,Account & transactions,NaN
433,434,Customer - Joint Account - Flag,NaN,Account & transactions,NaN


In [211]:
df_features["Category_labs"].unique()

array([nan, 'Name', 'DOB', 'POB', 'ID', 'Deceased', 'Occupation',
       'Occupation ', 'Spouse', 'Mother', 'Father', 'Brother', 'Sister',
       'Son', 'Daughter', 'Son in law', 'Daughter in law',
       'Father in law', 'Mother in law', 'Sister in law',
       'Brother in law', 'Grandparents', 'Grandchildren', 'Uncle', 'Aunt',
       'Cousin', 'Driver', 'Bodyguard', 'Secretaries', 'Business',
       'Ex-Husband', 'Account & transactions'], dtype=object)

In [212]:
related_party_list=['Spouse', 'Mother', 'Father', 'Brother', 'Sister',
       'Son', 'Daughter', 'Son in law', 'Daughter in law',
       'Father in law', 'Mother in law', 'Sister in law',
       'Brother in law', 'Grandparents', 'Grandchildren', 'Uncle', 'Aunt',
       'Cousin', 'Driver', 'Bodyguard', 'Secretaries', 'Business',
       'Ex Husband']
related_party_list=[prt.lower() for prt in related_party_list]

In [213]:
df_features.rename(columns={"Standardized_Anchors_BRD_LABS":"Anchors"},inplace=True)

In [214]:
df_features.dropna(subset=["Anchors"],inplace=True)

In [215]:
df_features

,S.no,Anchors,BRD_FLAG,Categories,Category_labs
0,1,Word count difference,Y,Full Legal Name - Word Count,NaN
1,2,Name structure difference,Y,Full Legal Name - Name Structure,NaN
2,3,Name does not sound similiar,Y,Full Legal Name - Name Does Not Sound Similar,NaN
3,4,Native name charachter difference,Y,Full Legal Name - Native Name,Name
5,6,Date of birth (dob),Y,Date of Birth (DOB),DOB
...,...,...,...,...,...
430,431,Customer - Transaction - Number of transactions,NaN,Account & transactions,NaN
431,432,Customer - Transaction - Amount,NaN,Account & transactions,NaN
432,433,Customer - Transaction - Online / offline,NaN,Account & transactions,NaN
433,434,Customer - Joint Account - Flag,NaN,Account & transactions,NaN


In [216]:
# df_features=df_features.loc[:,["Topic No","Anchor - Topic","Anchor existing list","Presence"]]
df_features["Anchor_2"]=df_features["Anchors"].apply(lambda x: re.sub("\s*-\s*|\s*/\s*|,|\(|\)"," ",x))

In [217]:
df_features["Anchor_2"]=df_features["Anchor_2"].apply(lambda x: re.sub("match|mismatch","",x))

In [218]:
df_features["Anchor_2"].shape

(428,)

In [219]:
df_features["Anchor_2"]=df_features["Anchor_2"].str.lower()

In [17]:
patt_10=re.compile("(match|mismatch|false hit|true hit|material|non-material|non material)",re.IGNORECASE)

df["Match_Mismatch"]=df["V_COMMENTS"].apply(lambda x: re.findall(patt_10,x))

# patt_10=re.compile("(match|mismatch)",re.IGNORECASE)
# patt_11=re.compile("(false hit|true hit)",re.IGNORECASE)
# df["Match_Mismatch"]=df["V_COMMENTS"].str.extract(patt_10)
# df["True_False_hit"]=df["V_COMMENTS"].str.extract(patt_11)

df["V_COMMENTS"].loc[29]

'mismatch id number _ nric'

In [19]:
import ast
df["Tokens"]=df["Tokens"].apply(lambda x: ast.literal_eval(x))

# Pre-processing on Data

### Step -1 removing noisy data

In [20]:
import calendar
cal=list(calendar.month_name)
for i in list(calendar.month_name[1:]):
    cal.append(i[:3])

In [21]:
patt1=re.compile("Match ID and/or Full Legal Name",re.IGNORECASE)

# patt2=re.compile("Agreed with analyst recommendation|Agreed with analyst's recommendation|Agreed as per Maker comment|Agreed as per Maker's comment")
patt2=re.compile("(Agreed|agree) (to|with|as per) (analyst(('s)*)|maker(('s)*)) (comment|recommendation|is recommendation)",re.IGNORECASE)
patt3=re.compile("Unknown|false hit\.*|true hit\.*|mismatch\.*|match\.*|non-material|non material\.*|Rationale|Source",re.IGNORECASE)
patt4=re.compile("CSCDD7|CSCDD1|CSCDD4",re.IGNORECASE)
patt5=re.compile("SCC6|No further review is required by analyst|SC/DEL",re.IGNORECASE)
patt6=re.compile("https?://\S+|www\.\S+",re.IGNORECASE)
patt7=re.compile("(%s)"%"|".join(cal)[1:],re.IGNORECASE)
patt8=re.compile("\s*(Mohd|NORAINI|ABDULLAH|BINTI|system|legal|stated|ABD|mahdzir|bin|khalid|siti|hajar|mohd|sallehudin|mohd|saddam|inie|binti|abdullah|noraini|norliza|binti|othman)\s*",re.IGNORECASE)
# patt_11=re.compile("",re.IGNORECASE)

In [22]:
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS"].apply(lambda x: re.sub("\n"," ",x)) #1
# df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt_11,"nonmaterial",x))
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("Name – Name","Name",x))    #2
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("Name_Name","Name",x))      #3
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("\s*_\s*|\s*-\s*"," ",x)) #4,5
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt1)==False] #7
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt2,"",x)) #8
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt3," ",x))  #  9
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt4)==False]        # 10
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt5)==False]       #11
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt6,"",x)) #15
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("[^\w\s\.]|\d"," ",x))
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("\d*","",x))    #date, year, accounts 6
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt7,"",x))    # month 6
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt8," ",x)) # 17


In [23]:
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].str.lower()

In [24]:
# import nltk
# nltk.download('punkt')

In [25]:
def remove_stopwords(document):
    "Return the articles after remvoing stopwords"
    article_tokens = word_tokenize(document) 
    filtered_article = [word for word in article_tokens if not word in stop_words] 
    return " ".join(filtered_article)

In [26]:
stop_words = set(stopwords.words('english')) 

In [27]:
#removing stopwords
df.V_COMMENTS_PROCESSED=df.V_COMMENTS_PROCESSED.apply(lambda x: remove_stopwords(x))

In [28]:
df["V_COMMENTS_TOKENS"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: word_tokenize(x))

In [29]:
for i,row in df.iterrows():
    if "." in row["V_COMMENTS_TOKENS"]:
        row["V_COMMENTS_TOKENS"].remove(".")

In [220]:

df_features["Anchor_2"]=df_features["Anchor_2"].apply(lambda x: remove_stopwords(x))
df_features["Anchor_Tokens"]=df_features["Anchor_2"].apply(lambda x:word_tokenize(x) )

In [42]:
# l1=df["V_COMMENTS_TOKENS"].to_list()
l2=df_features["Anchor_Tokens"].to_list()

In [32]:
df["V_COMMENTS_3"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: sent_tokenize(x))


In [33]:
df["V_COMMENTS_3"]

0                                          [id number nric]
1                                          [id number nric]
2                                          [year birth yob]
3                                          [id number nric]
4                          [full name â name sound similar]
                                ...                        
221569    [name nric date birth profile customer name nr...
221570    [name nric date birth profile customer per hos...
221571    [id number nric ., watch person bank customer ...
221572    [id number nric ., watch person bank customer ...
221573    [profile adverse news reported detained punjab...
Name: V_COMMENTS_3, Length: 221527, dtype: object

In [34]:
df["V_COMMENTS_3"]=df["V_COMMENTS_3"].apply(lambda x:[i.replace(".","") for i in x])

In [35]:
df[["V_COMMENTS","V_COMMENTS_3"]]

,V_COMMENTS,V_COMMENTS_3
0,mismatch id number_nric,[id number nric]
1,mismatch id number_nric\n,[id number nric]
2,mismatch year of birth (yob),[year birth yob]
3,mismatch id number_nric,[id number nric]
4,mismatch full legal name â€“ name does not sou...,[full name â name sound similar]
...,...,...
221569,"mismatch - name, nric, date of birth, profile,...",[name nric date birth profile customer name nr...
221570,"mismatch - name, nric, date of birth, profile,...",[name nric date birth profile customer per hos...
221571,mismatch id number_nric. watch person (noraini...,"[id number nric , watch person bank customer n..."
221572,mismatch id number_nric. watch person (noraini...,"[id number nric , watch person bank customer n..."


In [36]:
# ind1=df[df["V_COMMENTS_3"].apply(lambda x: np.any([len(i)>50 for i in x]))][["V_COMMENTS_3","V_COMMENTS"]].index

In [37]:
df["V_COMMENTS_3"]=df["V_COMMENTS_3"].apply(lambda x:[word_tokenize(sent) for sent in x ])

In [38]:
df["V_COMMENTS_3"]

0                                      [[id, number, nric]]
1                                      [[id, number, nric]]
2                                      [[year, birth, yob]]
3                                      [[id, number, nric]]
4                   [[full, name, â, name, sound, similar]]
                                ...                        
221569    [[name, nric, date, birth, profile, customer, ...
221570    [[name, nric, date, birth, profile, customer, ...
221571    [[id, number, nric], [watch, person, bank, cus...
221572    [[id, number, nric], [watch, person, bank, cus...
221573    [[profile, adverse, news, reported, detained, ...
Name: V_COMMENTS_3, Length: 221527, dtype: object

In [48]:
l1=df["V_COMMENTS_3"].apply(lambda x: [ j for i in x for j in i]).to_list()

In [49]:
l1.extend(l2)

In [ ]:
# l2=[]
# for l in df.loc[ind1,"V_COMMENTS_3"].head():
#     l3=[]
#     for i in l:
#         if len(i)>50:
#               inputs=tokenizer.encode(i,return_tensors='pt', max_length=512, truncation=True)
#               output = model.generate(inputs, min_length=15, max_length=len(i))
#               summary=tokenizer.decode(output[0])
#               l3.append(summary)
#         else:
#               l3.append(i)
#     l2.append(l3)

In [50]:
model1= gensim.models.Word2Vec(l1, min_count=1,
                                vector_size=50, window=5,sg=0)

In [51]:
df["Vect"]=df["V_COMMENTS_3"].apply(lambda x:[np.sum(np.array([model1.wv[i] for i in sent]),axis=0)/(len(sent)) for sent in x])

C:\Users\MBBLABS\AppData\Local\Temp\1\ipykernel_7684\1885035559.py:1: RuntimeWarning: invalid value encountered in double_scalars
  df["Vect"]=df["V_COMMENTS_3"].apply(lambda x:[np.sum(np.array([model1.wv[i] for i in sent]),axis=0)/(len(sent)) for sent in x])


In [54]:
df["Vect"]

0         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
1         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
2         [[-1.4059262, -1.2774773, 2.4419215, 0.2973410...
3         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
4         [[-0.013578136, -0.056594092, 1.2331814, 0.540...
                                ...                        
221569    [[-1.1782211, -1.7151439, 0.25849703, 0.581718...
221570    [[-1.080064, -1.7389482, 0.60544246, 0.8949438...
221571    [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
221572    [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
221573    [[0.14264989, -0.6006267, -3.411338, 1.2640637...
Name: Vect, Length: 221527, dtype: object

In [55]:
df_features["Vect 2"]=df_features["Anchor_Tokens"].apply(lambda x:np.sum(np.array([model1.wv[i] for i in x]),axis=0)/(len(x)))

In [56]:
df_features[["Vect 2","Anchor_Tokens"]]

,Vect 2,Anchor_Tokens
0,"[1.6281395, 0.37920544, 1.8039938, -1.0749658,...","[word, count, difference]"
1,"[1.380865, 0.22409028, 2.2350354, -1.9758564, ...","[name, structure, difference]"
2,"[-1.1274933, -0.19183542, 1.5031052, -0.642332...","[name, sound, similiar]"
3,"[1.18698, 0.80039066, 0.9883921, -2.805416, -0...","[native, name, charachter, difference]"
5,"[-2.0797102, -2.0875318, 3.629677, -0.4107891,...","[date, birth, dob]"
...,...,...
430,"[-2.5189621, -2.4859726, 0.057923406, -0.88686...","[customer, transaction, number, transactions]"
431,"[-1.7992836, -1.532343, 0.6821043, -0.34206367...","[customer, transaction, amount]"
432,"[-1.3401552, -0.8010278, -0.1553929, 0.2737053...","[customer, transaction, online, offline]"
433,"[-0.85839814, -2.225723, -0.44810647, -0.41440...","[customer, joint, account, flag]"


In [57]:
df["Vect_copy"]=df["Vect"].apply(lambda x: [ np.zeros((1,50))[0] if np.any(i==np.nan) or np.any(pd.isna(i)) else i for i in x])

In [58]:
df["Vect_copy"]

0         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
1         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
2         [[-1.4059262, -1.2774773, 2.4419215, 0.2973410...
3         [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
4         [[-0.013578136, -0.056594092, 1.2331814, 0.540...
                                ...                        
221569    [[-1.1782211, -1.7151439, 0.25849703, 0.581718...
221570    [[-1.080064, -1.7389482, 0.60544246, 0.8949438...
221571    [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
221572    [[-1.9399929, -3.713267, 1.9372615, 0.9949212,...
221573    [[0.14264989, -0.6006267, -3.411338, 1.2640637...
Name: Vect_copy, Length: 221527, dtype: object

In [ ]:
def f(x):
      l=[]
      for sent_vec in x:
          l.append(sorted([(cosine_similarity([sent_vec],[feat])[0][0],feat_name )for feat,feat_name in zip(df_features["Vect 2"],df_features["Anchor_Topic_Tokens"])],key=lambda x: x[0],reverse=True)[:3] )

      return l
for i,vecs in zip(df.index,df["Vect_copy"]):

    df["Features_present"].loc[i]=f(vecs)
# df[df.loc[:,"Vect"].apply(lambda x:np.array(x).shape==(9,))]["Vect"][24230]
# df["Vect_copy"]=df["Vect"].apply(lambda x:[np.zeros((1,50) ) if np.any(pd.isna(i)) else i for i in x  ])

In [59]:
import warnings
warnings.filterwarnings("ignore")

In [167]:
df.sample(frac=1)

,V_COMMENTS,Counts,V_COMMENTS_2,Tokens,Match_Mismatch,V_COMMENTS_PROCESSED,V_COMMENTS_TOKENS,V_COMMENTS_3,Vect,Vect_copy
10114,mismatch occupation- customer works as a manuf...,3,mismatch occupation customer works manufacturi...,"[mismatch, occupation, customer, work, manufac...",[mismatch],occupation customer works manufacturing labour...,"[occupation, customer, works, manufacturing, l...","[[occupation, customer, works, manufacturing, ...","[[2.374195, -0.02945184, -0.56837445, -0.17594...","[[2.374195, -0.02945184, -0.56837445, -0.17594..."
168188,mismatch year of birth (yob) watch person yob ...,1,mismatch year birth yob watch person yob infer...,"[mismatch, year, birth, yob, watch, person, yo...",[mismatch],year birth yob watch person yob inferred per w...,"[year, birth, yob, watch, person, yob, inferre...","[[year, birth, yob, watch, person, yob, inferr...","[[-1.4025713, -1.0360293, 0.32436377, 0.767833...","[[-1.4025713, -1.0360293, 0.32436377, 0.767833..."
75210,mismatch occupation. watch person was a member...,1,mismatch occupation watch person member manda...,"[mismatch, occupation, watch, person, member, ...",[mismatch],occupation . watch person member mandalay regi...,"[occupation, watch, person, member, mandalay, ...","[[occupation], [watch, person, member, mandala...","[[2.2649877, -1.2260165, -0.5555399, 1.8754185...","[[2.2649877, -1.2260165, -0.5555399, 1.8754185..."
125771,mismatch date of birth: customer (1973) and wa...,1,mismatch date birth customer watchperson dob p...,"[mismatch, date, birth, customer, watchperson,...",[mismatch],date birth customer watchperson dob per refini...,"[date, birth, customer, watchperson, dob, per,...","[[date, birth, customer, watchperson, dob, per...","[[-0.7484134, -1.7372613, 0.4622757, 1.0987536...","[[-0.7484134, -1.7372613, 0.4622757, 1.0987536..."
25406,mismatch profile. customer work as agricultura...,1,mismatch profile customer work agricultural f...,"[mismatch, profile, customer, work, agricultur...",[mismatch],profile . customer work agricultural forestry ...,"[profile, customer, work, agricultural, forest...","[[profile], [customer, work, agricultural, for...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,..."
...,...,...,...,...,...,...,...,...,...,...
183506,mismatch profile/location. \nwatchperson serve...,1,mismatch profilelocation watchperson serves m...,"[mismatch, profilelocation, watchperson, serve...",[mismatch],profile location . watchperson serves member s...,"[profile, location, watchperson, serves, membe...","[[profile, location], [watchperson, serves, me...","[[-0.4652835, 0.14600158, -2.8502443, 2.685030...","[[-0.4652835, 0.14600158, -2.8502443, 2.685030..."
129189,mismatch occupation :\nwatchperson is member o...,1,mismatch occupation watchperson member supervi...,"[mismatch, occupation, watchperson, member, su...",[mismatch],occupation watchperson member supervisory boar...,"[occupation, watchperson, member, supervisory,...","[[occupation, watchperson, member, supervisory...","[[-0.7871582, -0.39871827, -0.3383193, -0.1386...","[[-0.7871582, -0.39871827, -0.3383193, -0.1386..."
171811,false hit- jul 2019 -wp has registered by coun...,1,wp registered counter terrorism department pak...,"[wp, register, counter, terrorism, department,...",[false hit],wp registered counter terrorism department pak...,"[wp, registered, counter, terrorism, departmen...","[[wp, registered, counter, terrorism, departme...","[[0.54276377, -0.31742063, -3.9231071, 1.45457...","[[0.54276377, -0.31742063, -3.9231071, 1.45457..."
104541,mismatch profile.\ncustomer was born on 25/10/...,1,mismatch profile customer born watch persons...,"[mismatch, profile, customer, bear, watch, per...",[mismatch],profile . customer born . watch person dhter y...,"[profile, customer, born, ., watch, person, dh...","[[profile], [customer, born], [watch, person, ...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226

In [168]:
df_samp=df.sample(frac=1)

In [169]:
# df_samp=pd.concat([df_samp,df.tail(100)],axis=0,join="outer")


In [170]:
df_samp.reset_index(inplace=True)

### Cosine Similarity

In [171]:
df_samp["Features_present_cosine"]=""
def f(x,y):
      l=[]
      for sent_vec,sent in zip(x,y):
          l.append(sorted([[cosine_similarity([sent_vec],[feat])[0][0],feat_name, sent] for feat,feat_name in zip(df_features["Vect 2"],df_features["Anchor_Tokens"]) if cosine_similarity([sent_vec],[feat])[0][0] >0.5],key=lambda x: x[0],reverse=True) )

      return l
for i,vecs,comm in zip(df_samp.index,df_samp["Vect_copy"],df_samp["V_COMMENTS_3"]):

    df_samp["Features_present_cosine"].loc[i]=f(vecs,comm)
# df_samp[df_samp.loc[:,"Vect"].apply(lambda x:np.array(x).shape==(9,))]["Vect"][24230]
# df_samp["Vect_copy"]=df_samp["Vect"].apply(lambda x:[np.zeros((1,50) ) if np.any(pd.isna(i)) else i for i in x  ])

KeyboardInterrupt: 

In [174]:
df_samp_2=df_samp[df_samp["Features_present_cosine"].apply(lambda x: np.any(pd.notna(x)))]

In [176]:
df_samp_2.head()

,index,V_COMMENTS,Counts,V_COMMENTS_2,Tokens,Match_Mismatch,V_COMMENTS_PROCESSED,V_COMMENTS_TOKENS,V_COMMENTS_3,Vect,Vect_copy,Features_present_cosine
0,118129,"mismatch profile, according to apex (sa 112763...",1,mismatch profile according apex sa customers a...,"[mismatch, profile, accord, apex, sa, customer...",[mismatch],profile according apex sa customers account ac...,"[profile, according, apex, sa, customers, acco...","[[profile, according, apex, sa, customers, acc...","[[-0.58316666, -2.2768817, -0.5916827, 1.42259...","[[-0.58316666, -2.2768817, -0.5916827, 1.42259...","[[[0.7855438, ['customer', 'account', 'active'..."
1,127687,mismatch occupation :\nwatchperson is member o...,1,mismatch occupation watchperson member supervi...,"[mismatch, occupation, watchperson, member, su...",[mismatch],occupation watchperson member supervisory boar...,"[occupation, watchperson, member, supervisory,...","[[occupation, watchperson, member, supervisory...","[[-0.28188333, -0.58418506, -0.48625526, -0.36...","[[-0.28188333, -0.58418506, -0.48625526, -0.36...","[[], [[0.6807698, ['watch', 'person', 'holding..."
2,123218,mismatch profile. customer who is primary scho...,1,mismatch profile customer priy school early c...,"[mismatch, profile, customer, priy_school_earl...",[mismatch],profile . customer priy school early childhood...,"[profile, customer, priy, school, early, child...","[[profile], [customer, priy, school, early, ch...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[[0.8977755, ['customer', 'occupation', 'biog..."
3,149598,mismatch biography/occupation â€“ analystâ€™s ...,1,mismatch biographyoccupation analysts justific...,"[mismatch, analysts_justification, watchperson...",[mismatch],biography occupation â analystâ justification ...,"[biography, occupation, â, analystâ, justifica...","[[biography, occupation, â, analystâ, justific...","[[-0.18106897, -0.9852555, -0.969547, 0.559660...","[[-0.18106897, -0.9852555, -0.969547, 0.559660...","[[[0.63438046, ['watch', 'person', 'secretarie..."
4,221214,mismatch profile. in nov 2018 - reportedly sen...,1,mismatch profile reportedly sentenced high co...,"[mismatch, profile, reportedly, sentence, high...",[mismatch],profile . reportedly sentenced high court deat...,"[profile, reportedly, sentenced, high, court, ...","[[profile], [reportedly, sentenced, high, cour...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[[0.8977755, ['customer', 'occupation', 'biog..."


In [177]:
df_samp_2["Features_present_cosine"]=df_samp_2["Features_present_cosine"].apply(lambda x: [ j for i in x for j in i])

In [178]:
df_samp_2["Features_present_cosine_top_5"]=df_samp_2["Features_present_cosine"].apply(lambda x: sorted(x,key=lambda x:x[0],reverse=True)[:5])

In [122]:
### The end
# df_samp["Features_present_cosine_top_5"]=df_samp["Features_present_cosine"].apply(lambda x: x[:5])

In [179]:
df_samp_2["Features_present_cosine_actual_anchors"]=df_samp_2["Features_present_cosine_top_5"].apply(lambda x: [i[1] for i in x]).apply(lambda x: [ " ".join(i) for i in x])

In [180]:
df_samp_2["Features_present_cosine_actual_anchors"]=df_samp_2["Features_present_cosine_actual_anchors"].apply(lambda x: list(set(x)))

In [221]:
df_samp_2["Feat_Categories"]=""

In [222]:
categories=[]
for i in df_features[["Anchor_2","Categories"]].values:
    categories.append(list(i))

In [223]:
categories

[['word count difference', 'Full Legal Name - Word Count'],
 ['name structure difference', 'Full Legal Name - Name Structure'],
 ['name sound similiar', 'Full Legal Name - Name Does Not Sound Similar'],
 ['native name charachter difference', 'Full Legal Name - Native Name'],
 ['date birth dob', 'Date of Birth (DOB)'],
 ['year birth yob age age date', 'Age/ Year of Birth (YOB)'],
 ['match place birth', 'Place of Birth'],
 ['id number nric', 'Identification Number - NRIC'],
 ['id citizenship number', 'Identification Number - Citizenship Number'],
 ['watch person deceased date', 'Deceased Status'],
 ['watch person age deceased', 'Deceased Status'],
 ['watch person deceased flag', 'Deceased Status'],
 ['customer deceased flag', 'Deceased Status'],
 ['customer deceased date', 'Deceased Status'],
 ['gender', 'Gender'],
 ['father name', "Father's Name "],
 ['customer occupation biography profile', 'Biography/ Occupation'],
 ['customer occupation biography profile', 'Biography/ Occupation'],
 

In [224]:
for ind,anch in zip(df_samp_2.index,df_samp_2["Features_present_cosine_actual_anchors"]):
    l=[]
    for x in anch:
        l.append([(x,i[1])  for i in categories if x in i][0])
    df_samp_2["Feat_Categories"].loc[ind]=l

In [225]:
df_samp_2["Feat_Categories"]

0         [(customer account active flag, Account & tran...
1         [(watch person holding high position flag, PEP...
2         [(watch person uncle occupation profile, Relat...
3         [(watch person driver occupation profile, Rela...
4         [(customer transaction online offline, Account...
                                ...                        
221522                                                   []
221523                                                   []
221524                                                   []
221525                                                   []
221526                                                   []
Name: Feat_Categories, Length: 220342, dtype: object

In [226]:
df_samp_2["V_COMMENTS_PROCESSED"]

0         profile according apex sa customers account ac...
1         occupation watchperson member supervisory boar...
2         profile . customer priy school early childhood...
3         biography occupation â analystâ justification ...
4         profile . reportedly sentenced high court deat...
                                ...                        
221522    watchperson reprimanded penalty myr imposed ms...
221523                       date birth dob watchperson dob
221524    profile watchperson member national people con...
221525    profile . customer manufacturing labourers lay...
221526    profile . based uco customer works manufacturi...
Name: V_COMMENTS_PROCESSED, Length: 220342, dtype: object

In [227]:
df_samp_2["Categories"]=df_samp_2["Feat_Categories"].apply(lambda x: list(set([i[1] for i in x])))

In [228]:
df_samp_2["V_COMMENTS_PROCESSED"]


0         profile according apex sa customers account ac...
1         occupation watchperson member supervisory boar...
2         profile . customer priy school early childhood...
3         biography occupation â analystâ justification ...
4         profile . reportedly sentenced high court deat...
                                ...                        
221522    watchperson reprimanded penalty myr imposed ms...
221523                       date birth dob watchperson dob
221524    profile watchperson member national people con...
221525    profile . customer manufacturing labourers lay...
221526    profile . based uco customer works manufacturi...
Name: V_COMMENTS_PROCESSED, Length: 220342, dtype: object

In [229]:
df_samp_2["Rel_flag"]=df_samp_2[df_samp_2["Categories"].apply(lambda x: "Related Parties" in x)]["V_COMMENTS_TOKENS"].apply(lambda x:sum([1 if tok in x else 0 for tok in related_party_list])>0)

In [230]:
df_samp_2[df_samp_2["Rel_flag"]==False]["Categories"].apply(lambda x: x.remove("Related Parties"))

0        None
1        None
2        None
3        None
4        None
         ... 
54786    None
54787    None
54791    None
54792    None
54793    None
Name: Categories, Length: 37696, dtype: object

In [231]:
df_samp_2["Categories"]

0                                [Account & transactions]
1                                                   [PEP]
2                                 [Biography/ Occupation]
3                                                      []
4         [Account & transactions, Biography/ Occupation]
                               ...                       
221522                                                 []
221523                                                 []
221524                                                 []
221525                                                 []
221526                                                 []
Name: Categories, Length: 220342, dtype: object

In [ ]:
# df_samp[["Categories","Counts"]].apply(lambda x: [ for i in x],axis=1)

In [ ]:
# df_samp[""]

In [149]:
df_features["Categories"][-2].unique()


array(['Full Legal Name - Word Count', 'Full Legal Name - Name Structure',
       'Full Legal Name - Name Does Not Sound Similar',
       'Full Legal Name - Native Name', 'Date of Birth (DOB)',
       'Age/ Year of Birth (YOB)', 'Place of Birth',
       'Identification Number - NRIC',
       'Identification Number - Citizenship Number', 'Deceased Status',
       'Gender', "Father's Name ", 'Biography/ Occupation',
       'Identification Number - Passport Number', 'Name', 'DOB', 'ID',
       'Related Parties', 'PEP', 'UNKOWN/Miscellanoeous', 'Securities',
       'Crime', 'Account & transactions',
       'Proof of customer location elsewhere/ profile/ present in the bank during the period when the aderse news took place',
       'Account & Transaction', 'Account &Transaction'], dtype=object)

In [232]:
df_samp_2[["V_COMMENTS","Counts","Match_Mismatch","Feat_Categories","Categories"]].to_csv("Feature_extraction_cosine_complete_data_train.csv")

In [150]:
df_stats=pd.DataFrame({"Categories":df_features["Categories"].unique()[:-2],})

In [151]:
df_stats["Counts"]=""

In [152]:
for i,rows in df_stats.iterrows():
    df_stats.loc[i,"Counts"]=df_samp[df_samp["Categories"].apply(lambda x: rows["Categories"] in x)]["Counts"].sum()


In [153]:
max=df_samp["Counts"].sum()
df_stats["Percentage"]=df_stats["Counts"].apply(lambda x: x/ max)

In [154]:
max

1767

In [156]:
df_stats.to_csv("Categories_stats_rem_rel_part.csv")

### Jaccard

In [102]:
def Jaccard_Similarity(words_doc1, words_doc2): 
    
    # List the unique words in a document
#     words_doc1 = set(doc1.lower().split()) 
#     words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)
#     print(len(intersection))
    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
#     print(len(union))
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [126]:
resu=Jaccard_Similarity({'name', 'similiar', 'does', 'sound', 'not'},{'name', 'does', 'sound', 'not', 'similar', 'full'})
resu

0.5714285714285714

In [103]:
df_features["Anchor_Tokens"]

0                          [word, count, difference]
1                      [name, structure, difference]
2                            [name, sound, similiar]
3             [native, name, charachter, difference]
5                                 [date, birth, dob]
                           ...                      
430    [customer, transaction, number, transactions]
431                  [customer, transaction, amount]
432         [customer, transaction, online, offline]
433                 [customer, joint, account, flag]
434            [customer, last, transaction, amount]
Name: Anchor_Tokens, Length: 428, dtype: object

In [104]:
# df_samp_features["Anchor_Tokens"]

df_samp["Features_present_J"]=""
def f(x):
      l=[]
      for sent_tok in x:
#           l.append(sorted([(Jaccard_Similarity(sent_tok,feat)[0][0],feat_name )for feat,feat_name in df_samp_features["Anchor_Tokens"]) if cosine_similarity([sent_vec],[feat])[0][0] >0.5],key=lambda x: x[0],reverse=True) )
          l.append(sorted([[Jaccard_Similarity(set(sent_tok),set(feat)),feat,sent_tok]  for feat in df_features["Anchor_Tokens"] ],key=lambda x: x[0],reverse=True) )
      return l
for i,vecs in zip(df_samp.index,df_samp["V_COMMENTS_3"]):

    df_samp["Features_present_J"].loc[i]=f(vecs)

In [105]:
df_samp[["V_COMMENTS","Features_present_J"]]

,V_COMMENTS,Features_present_J
0,mismatch dob. customer dob does not fall withi...,"[[[0.3333333333333333, ['date', 'birth', 'dob'..."
1,mismatch profile - customer as manufacturing l...,"[[[0.11764705882352941, ['customer', 'occupati..."
2,mismatch profile. customer who is manufacturin...,"[[[0.25, ['customer', 'occupation', 'biography..."
3,false hit. watchperson is son of pep who is a ...,"[[[0.15384615384615385, ['watch', 'person', 's..."
4,mismatch profile based on host the customer is...,"[[[0.13333333333333333, ['customer', 'occupati..."
...,...,...
195,"mismatch - name, nric, date of birth, profile,...","[[[0.17391304347826086, ['watch', 'person', 'p..."
196,"mismatch - name, nric, date of birth, profile,...","[[[0.23529411764705882, ['watch', 'person', 'p..."
197,mismatch id number_nric. watch person (noraini...,"[[[1.0, ['id', 'number', 'nric'], ['id', 'numb..."
198,mismatch id number_nric. watch person (noraini...,"[[[1.0, ['id', 'number', 'nric'], ['id', 'numb..."


In [106]:
df_samp["Features_present_J"]=df_samp["Features_present_J"].apply(lambda x:  [ j for i in x for j in i])

In [107]:
df_samp["Features_present_J"]

0      [[0.3333333333333333, [date, birth, dob], [dob...
1      [[0.11764705882352941, [customer, occupation, ...
2      [[0.25, [customer, occupation, biography, prof...
3      [[0.15384615384615385, [watch, person, son, pe...
4      [[0.13333333333333333, [customer, occupation, ...
                             ...                        
195    [[0.17391304347826086, [watch, person, pep, da...
196    [[0.23529411764705882, [watch, person, pep, da...
197    [[1.0, [id, number, nric], [id, number, nric]]...
198    [[1.0, [id, number, nric], [id, number, nric]]...
199    [[0.05555555555555555, [customer, occupation, ...
Name: Features_present_J, Length: 200, dtype: object

In [108]:
df_features.columns

Index(['S.no', 'Anchors', 'BRD_FLAG', 'Categories', 'Category_labs',
       'Anchor_2', 'Anchor_Tokens', 'Vect 2'],
      dtype='object')

In [109]:
df_samp["Features_present_Top_5"]=df_samp["Features_present_J"].apply(lambda x: sorted(x,key=lambda x:x[0],reverse=True)[:5])

In [132]:
# df_samp["Features_present_Top_5"]=df_samp["Features_present_J"].apply(lambda x: x[0:5])

In [110]:
df_samp["Features_actual_J_anchors"]=df_samp["Features_present_Top_5"].apply(lambda x:[i[1] for i in x])

In [111]:
df_samp["Features_actual_J_anchors"]=df_samp["Features_actual_J_anchors"].apply(lambda x: [" ".join(i) for i in x])

In [112]:
df_samp["Features_actual_J_anchors"]=df_samp["Features_actual_J_anchors"].apply(lambda x: list(set(x)))

In [113]:
df_samp["Feat_Categories_J"]=""
for ind,anch in zip(df_samp.index,df_samp["Features_actual_J_anchors"]):
    l=[]
    for x in anch:
        l.append([(x,i[1])  for i in categories if x in i][0])
    df_samp["Feat_Categories_J"].loc[ind]=l

In [114]:
df_samp["Categories_J"]=df_samp["Feat_Categories_J"].apply(lambda x: list(set([i[1] for i in x])))

In [116]:
df_samp[["V_COMMENTS","Counts","Match_Mismatch","V_COMMENTS_3","Features_present_Top_5","Features_actual_J_anchors","Feat_Categories_J","Categories_J"]].to_csv("Feature_extraction_nov_Jaccard_sim_6.csv")

In [101]:
df_samp[["V_COMMENTS","Counts","Match_Mismatch","V_COMMENTS_3","Features_present_cosine","Features_present_cosine_top_5","Features_present_cosine_actual_anchors","Feat_Categories","Categories"]].to_csv("Feature_extraction_nov_Jaccard_sim_5.csv")

In [234]:
pip install bertopic

Could not fetch URL https://pypi.org/simple/bertopic/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/bertopic/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))) - skippingNote: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement bertopic (from versions: none)
ERROR: No matching distribution found for bertopic



Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))) - skipping


In [233]:
# df_samp_2
from bertopic import BERTopic
docs = list(df.reviews.values)
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

,index,V_COMMENTS,Counts,V_COMMENTS_2,Tokens,Match_Mismatch,V_COMMENTS_PROCESSED,V_COMMENTS_TOKENS,V_COMMENTS_3,Vect,Vect_copy,Features_present_cosine,Features_present_cosine_top_5,Features_present_cosine_actual_anchors,Feat_Categories,Categories,Rel_flag
0,118129,"mismatch profile, according to apex (sa 112763...",1,mismatch profile according apex sa customers a...,"[mismatch, profile, accord, apex, sa, customer...",[mismatch],profile according apex sa customers account ac...,"[profile, according, apex, sa, customers, acco...","[[profile, according, apex, sa, customers, acc...","[[-0.58316666, -2.2768817, -0.5916827, 1.42259...","[[-0.58316666, -2.2768817, -0.5916827, 1.42259...","[[0.7855438, [customer, account, active, flag]...","[[0.7855438, [customer, account, active, flag]...","[customer account active flag, customer last t...","[(customer account active flag, Account & tran...",[Account & transactions],False
1,127687,mismatch occupation :\nwatchperson is member o...,1,mismatch occupation watchperson member supervi...,"[mismatch, occupation, watchperson, member, su...",[mismatch],occupation watchperson member supervisory boar...,"[occupation, watchperson, member, supervisory,...","[[occupation, watchperson, member, supervisory...","[[-0.28188333, -0.58418506, -0.48625526, -0.36...","[[-0.28188333, -0.58418506, -0.48625526, -0.36...","[[0.6807698, [watch, person, holding, high, po...","[[0.6807698, [watch, person, holding, high, po...","[watch person holding high position flag, watc...","[(watch person holding high position flag, PEP...",[PEP],False
2,123218,mismatch profile. customer who is primary scho...,1,mismatch profile customer priy school early c...,"[mismatch, profile, customer, priy_school_earl...",[mismatch],profile . customer priy school early childhood...,"[profile, customer, priy, school, early, child...","[[profile], [customer, priy, school, early, ch...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[0.8977755, [customer, occupation, biography,...","[[0.8977755, [customer, occupation, biography,...","[watch person uncle occupation profile, watch ...","[(watch person uncle occupation profile, Relat...",[Biography/ Occupation],False
3,149598,mismatch biography/occupation â€“ analystâ€™s ...,1,mismatch biographyoccupation analysts justific...,"[mismatch, analysts_justification, watchperson...",[mismatch],biography occupation â analystâ justification ...,"[biography, occupation, â, analystâ, justifica...","[[biography, occupation, â, analystâ, justific...","[[-0.18106897, -0.9852555, -0.969547, 0.559660...","[[-0.18106897, -0.9852555, -0.969547, 0.559660...","[[0.63438046, [watch, person, secretaries, occ...","[[0.63438046, [watch, person, secretaries, occ...","[watch person driver occupation profile, watch...","[(watch person driver occupation profile, Rela...",[],False
4,221214,mismatch profile. in nov 2018 - reportedly sen...,1,mismatch profile reportedly sentenced high co...,"[mismatch, profile, reportedly, sentence, high...",[mismatch],profile . reportedly sentenced high court deat...,"[profile, reportedly, sentenced, high, court, ...","[[profile], [reportedly, sentenced, high, cour...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[2.1939108, -2.2496226, -3.368641, 2.6579401,...","[[0.8977755, [customer, occupation, biography,...","[[0.8977755, [customer, occupation, biography,...","[customer transaction online offline, customer...","[(customer transaction online offline, Account...","[Account & transactions, Biography/ Occupation]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221522,99790,false hit. watchperson was reprimanded and pen...,1,watchperson reprimanded penalty myr imposed ms...,"[watchperson, reprimanded_penalty_myr, imposed...",[false hit],watchperson reprimanded penalty myr imposed ms...,"[watchperson, reprimanded, penalty, myr, impos...","[[watchperson, reprimanded, penalty, myr, impo...","